In [1]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
import albumentations
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import scipy as sp
from scipy.special import softmax
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from functools import partial
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

C:\Users\wangz\anaconda3\envs\tf_gpu23-2\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


device(type='cuda')

In [2]:
# ====================================================
# PATH
# ====================================================
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'
# ====================================================
# test data
# ====================================================
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))

In [56]:
# ====================================================
# CFG for Resnext
# ====================================================
class CFG:
    debug=False
    image_size = 512
    num_workers=0 # 4
    model_name='resnext50_32x4d'
    size=512
    batch_size=32
    seed=1992
    num_classes=5
    target_col='label'
    resnext = 'resnext50_32x4d'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True

# ====================================================
# RANDOM SEED
# ====================================================
def seed_all(seed: int):
    if not seed:
        seed = 10

    print("[ Using Seed : ", seed, " ]")
    os.environ['PYTHONHASHSEED'] = str(seed)  # set PYTHONHASHSEED env var at fixed value
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed) # pytorch (both CPU and CUDA)
    np.random.seed(seed) # for numpy pseudo-random generator
    random.seed(seed) # set fixed value for python built-in pseudo-random generator
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
seed_all(seed=CFG.seed)

# ====================================================
# Dataset for Resnext
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image
    
# ====================================================
# Transforms for Resnext
# ====================================================
import albumentations as A
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
# ====================================================
# ResNext Model
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, model_arc, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name=model_arc, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, num_classes)
        
    def forward(self, input_neurons):
        # TODO: add dropout layers, or the likes.
        output_predictions = self.model(input_neurons)
        return output_predictions
    
# ====================================================
# inference
# ====================================================
def load_state(model_path):
    model = CustomResNext(CFG.model_name, CFG.num_classes, pretrained=False)
    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_path)['model'], strict=True)
        state_dict = torch.load(model_path)['model']
    except:  # multi GPU model_file
        state_dict = torch.load(model_path)['model']
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

    return state_dict


def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state)
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
        
    probs = np.concatenate(probs)
    return probs

[ Using Seed :  1992  ]


In [57]:
#for Resnext
MODEL_DIR = '../input/cassavaroots/'
model = CustomResNext(CFG.model_name, CFG.num_classes, pretrained=False)
states = [load_state(f'{MODEL_DIR}{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False,  num_workers=CFG.num_workers, pin_memory=True)
resnext_predictions = inference(model, states, test_loader, device)
resnext_predictions

100%|██████████| 1/1 [00:00<00:00,  5.97it/s]


array([[0.06168885, 0.15705398, 0.32266027, 0.07326578, 0.38533115]],
      dtype=float32)

In [ ]:
# array([[0.04188796, 0.04432843, 0.24820718, 0.03057373, 0.63500273]],
#       dtype=float32)

In [26]:
# ====================================================
# CFG for Resnext
# ====================================================
class CFG:
    debug=False
    num_workers=0#4
    model_name='resnext50_32x4d'
    image_size=512
    batch_size=1
    seed=1992
    num_classes=5
    target_col='label'
    resnext = 'resnext50_32x4d'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True
    
# ====================================================
# seed
# ====================================================

def seed_all(seed: int):
    if not seed:
        seed = 10

    print("[ Using Seed : ", seed, " ]")
    
    os.environ['PYTHONHASHSEED'] = str(seed)  # set PYTHONHASHSEED env var at fixed value
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed) # pytorch (both CPU and CUDA)
    np.random.seed(seed) # for numpy pseudo-random generator
    random.seed(seed) # set fixed value for python built-in pseudo-random generator
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
seed_all(seed=CFG.seed)
    
# ====================================================
# Dataset for efficientnet
# ====================================================
class CLDDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        
        image = image.astype(np.float32)
        image = image.transpose(2,0,1)
        if self.mode == 'test':
            return torch.tensor(image).float()
        else:
            return torch.tensor(image).float(), torch.tensor(row.label).float()    
        
    
# ====================================================
# EfficientNet Model
# ====================================================
class enet_v2(nn.Module):

    def __init__(self, backbone, out_dim, pretrained=False):
        super(enet_v2, self).__init__()
        self.enet = timm.create_model(backbone, pretrained=pretrained)
        in_ch = self.enet.classifier.in_features
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.classifier = nn.Identity()

    def forward(self, x):
        x = self.enet(x)
        x = self.myfc(x)
        return x    
    
    
class CustomEfficientNet(nn.Module):
    def __init__(self, config: type, pretrained: bool=True):
        super().__init__()
        self.config = config
        self.model = geffnet.create_model(
#             model_weight_path_folder=config.paths['model_weight_path_folder'],
            model_name=config.effnet,
            pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, config.num_classes)
        

    def forward(self, input_neurons):
        # TODO: add dropout layers, or the likes.
        output_predictions = self.model(input_neurons)
        return output_predictions
    
# ====================================================
# Helper functions for efficientnet
# ====================================================
def inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)

    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS

def tta_inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
            x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
            x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],0)
            x = x.view(-1, 3, CFG.image_size, CFG.image_size)
            logits = model(x)
            logits = logits.view(CFG.batch_size, 8, -1).mean(1)
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
            LOGITS.append(logits.cpu())

        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

#Transform for efficientnet
transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(CFG.image_size, CFG.image_size, p=1),
    albumentations.Resize(CFG.image_size, CFG.image_size),
    albumentations.Normalize()
])

[ Using Seed :  1992  ]


In [32]:
#for Efficientnet
enet_type = ['tf_efficientnet_b4_ns'] * 5
model_path = ['../input/cassavaroots/baseline_cld_fold0_epoch8_tf_efficientnet_b4_ns_512.pth', 
              '../input/cassavaroots/baseline_cld_fold1_epoch9_tf_efficientnet_b4_ns_512.pth', 
              '../input/cassavaroots/baseline_cld_fold2_epoch9_tf_efficientnet_b4_ns_512.pth',
              '../input/cassavaroots/baseline_cld_fold3_epoch5_tf_efficientnet_b4_ns_512.pth',
              '../input/cassavaroots/baseline_cld_fold4_epoch11_tf_efficientnet_b4_ns_512.pth']


#for efficientnet
test_dataset_efficient = CLDDataset(test, 'test', transform=transforms_valid)
test_loader_efficient = torch.utils.data.DataLoader(test_dataset_efficient, batch_size=CFG.batch_size, shuffle=False,  num_workers=CFG.num_workers)

efb4_predictions = []
for i in range(len(enet_type)):
    model = enet_v2(enet_type[i], out_dim=5)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i]))
    efb4_predictions += [tta_inference_func(test_loader_efficient)]
efb4_predictions = np.mean(efb4_predictions, axis=0)
efb4_predictions

100%|██████████| 1/1 [00:00<00:00, 10.34it/s]


array([[0.03165457, 0.02644922, 0.15103984, 0.00727853, 0.7835778 ]],
      dtype=float32)

In [35]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation

In [69]:
# ====================================================
# CFG for keras ef b3
# ====================================================
# batch_size = 4
image_size = 512
input_shape = (image_size, image_size, 3)
dropout_rate = 0.4
# classes_to_predict = sorted(training_df.label.unique())
n_ouput = 5

# ====================================================
# build the ef graph
# ====================================================
def build_graph(weights=None):
    data_augmentation_layers = tf.keras.Sequential(
        [
            layers.experimental.preprocessing.RandomCrop(height=image_size, width=image_size),
            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.25),
            layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
            layers.experimental.preprocessing.RandomContrast((0.2,0.2))
        ]
    )

    efficientnet = EfficientNetB3(#weights='../model/efficientnet-b3_noisy-student.h5', 
                                  include_top=False, 
                                  input_shape=input_shape, 
                                  drop_connect_rate=dropout_rate)

    inputs = Input(shape=input_shape)
    augmented = data_augmentation_layers(inputs)
    efficientnet = efficientnet(augmented)
    pooling = layers.GlobalAveragePooling2D()(efficientnet)
    dropout = layers.Dropout(dropout_rate)(pooling)
    outputs = Dense(n_ouput, activation="softmax")(dropout)
    model = Model(inputs=inputs, outputs=outputs)
    
    if weights!=None:
        model.load_weights(weights)

    return model


test_time_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

def scan_over_image(img_path, crop_size=512):
    '''
    Will extract 512x512 images covering the whole original image
    with some overlap between images
    '''
    
    img = Image.open(img_path)
    img_height, img_width = img.size
    img = np.array(img)
    
    y = random.randint(0,img_height-crop_size)
    x = random.randint(0,img_width-crop_size)

    x_img_origins = [0,img_width-crop_size]
    y_img_origins = [0,img_height-crop_size]
    img_list = []
    for x in x_img_origins:
        for y in y_img_origins:
            img_list.append(img[x:x+crop_size , y:y+crop_size,:])
  
    return np.array(img_list)

def predict_and_vote(models, image_filename, folder, TTA_runs=4):
    '''
    Run the model over 4 local areas of the given image,
    before making a decision depending on the most predicted
    disease.
    '''
    
    #apply TTA to each of the 4 images and sum all predictions for each local image
    localised_predictions = []
    local_image_list = scan_over_image(folder+image_filename)
    for local_image in local_image_list:
        duplicated_local_image = tf.convert_to_tensor(np.array([local_image for i in range(TTA_runs)]))
        augmented_images = test_time_augmentation_layers(duplicated_local_image)
        predictions = models[0].predict(augmented_images)
        for m in models[1:]:
            predictions += m.predict(augmented_images)
        predictions /= len(models)
        localised_predictions.append(np.mean(predictions, axis=0))
    
    #sum all predictions from all 4 images and retrieve the index of the highest value
    global_predictions = np.mean(np.array(localised_predictions),axis=0)
#     final_prediction = np.argmax(global_predictions)
    
    return global_predictions

def run_predictions_over_image_list(models, image_list, folder):
    predictions = []
    with tqdm(total=len(image_list)) as pbar:
        for image_filename in image_list:
            pbar.update(1)
            predictions.append(predict_and_vote(models, image_filename, folder))
    return np.array(predictions)

In [70]:
test_folder = '../input/cassava-leaf-disease-classification/test_images/'
model_folder = '../input/keras-efficientnet/'
model_paths = [f'{model_folder}{f}' for f in os.listdir(model_folder) if 'fold' in f]
models =  []
for m_p in model_paths:
    models.append(build_graph(m_p))
kefb3_predictions = run_predictions_over_image_list(models, test.image_id, test_folder)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.24s/it]


In [78]:
OUTPUT_DIR = './'
pred = (resnext_predictions + efb4_predictions + kefb3_predictions)/3
test['label'] = softmax(pred).argmax(1)
test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)